In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

data_dir = r"E:\Dataset\IK_pcb_L12_augmented"
train_dir = os.path.join(data_dir, "train")

# EfficientNetB3 default image size is 300x300
img_size = 300
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.8, 1.2],
    shear_range=0.2,
    channel_shift_range=20,
    validation_split=0.2,
)

train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


base_model = EfficientNetB3(weights='imagenet', include_top=False, input_tensor=tf.keras.Input(shape=(img_size, img_size, 3)))


#for layer in base_model.layers[:-20]:
#    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x) # Added dropout
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks
checkpoint_cb = ModelCheckpoint("IK_pcb_L12_augmented2.keras", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(factor=0.5, patience=5)

history = model.fit(
    train_set,
    epochs=50,
    validation_data=val_set,
    callbacks=[checkpoint_cb, early_stopping_cb, reduce_lr_cb]
)